In [ ]:
import numpy as np
from pydrake.all import (
    DirectCollocation,
    DiagramBuilder,
    Simulator,
    SnoptSolver,
    PiecewisePolynomial,
    LeafSystem_,
    TemplateSystem,
    BasicVector_,
    SolverOptions,
    CommonSolverOption,Variable,
    sin,cos,pow,sqrt
)
import matplotlib.pyplot as plt
from matplotlib import cm

from robot_arm import ArmDynamics, ArmPlant_, Arm2DVisualizer, simplify_exp, Jacobian, inv3, det3

from IPython.display import HTML, display, Image
import multiprocessing as mp
import os

In [ ]:
def plot_traj(arm_dynamics, x_traj, u_traj, v_traj):
    
    t = np.linspace(x_traj.start_time(), x_traj.end_time(), int(1e4))
    
    x_log = np.array([x_traj.value(i) for i in t]).T
    u_log = np.array([u_traj.value(i) for i in t]).T
    v_log = np.array([v_traj.value(i) for i in t]).T
    velocities = []
    for q in x_log.T:
        velocities.append(arm_dynamics.EvalKeypoint(arm_dynamics.drE, q).T[0])
    velocities = np.array(velocities) 

    fig, ax = plt.subplots(2, 2, figsize=(15,12), facecolor='white')
    # joint angles
    #ax[0,0].plot(t, x_log.T[:,0], label='$q_1$')
    #ax[0,0].plot(t, x_log.T[:,1], label='$q_2$')
    #ax[0,0].plot(t, x_log.T[:,2], label='$q_3$')
    #ax[0,0].set_title("Joint Positions")
    #ax[0,0].set_xlabel("t (s)")
    #ax[0,0].set_ylabel("$\theta$ (rad)")
    #ax[0,0].legend()

    #joint velocities
    ax[0,1].plot(t, x_log.T[:,3], label='$\dot{q_1}$')
    ax[0,1].plot(t, x_log.T[:,4], label='$\dot{q_3}$')
    ax[0,1].plot(t, x_log.T[:,5], label='$\dot{q_4}$')
    ax[0,1].set_title("Joint Velocities")
    ax[0,1].set_xlabel("t (s)")
    ax[0,1].set_ylabel("$\dot{\\theta}$ (rad/s)")
    ax[0,1].legend()

    ax[0,0].plot(t, v_log.T[:,0], label='$V_1$')
    ax[0,0].plot(t, v_log.T[:,1], label='$V_2$')
    ax[0,0].set_title("Motor Voltages")
    ax[0,0].set_xlabel("t (s)")
    ax[0,0].set_ylabel("Voltage (V)")
    ax[0,0].legend()
    
    ax[1,0].plot(t, u_log.T[:,0], label='$\\tau_1$')
    ax[1,0].plot(t, u_log.T[:,1], label='$V\tau_2$')
    ax[1,0].set_title("Motor Torques")
    ax[1,0].set_xlabel("t (s)")
    ax[1,0].set_ylabel("Torque (Nm)")
    ax[1,0].legend()

    ax[1,1].plot(t, velocities[:,0], label='$\dot{x}$')
    ax[1,1].plot(t, velocities[:,1], label='$\dot{y}$')
    ax[1,1].plot(t, np.sqrt(velocities[:,0]**2 +  velocities[:,1]**2), label='$\sqrt{\dot{x}^2+\dot{y}^2}$')
    ax[1,1].set_title("Endeffector velocity")
    ax[1,1].set_xlabel("t (s)")
    ax[1,1].set_ylabel("speed (m/s)")
    ax[1,1].legend()

    return fig


def visualize_arm_trajectory(arm_dynamics, x_traj, frame_rate=1 / 60.0, post_buffer = 2.0, show = True):
    builder = DiagramBuilder()
    plant = builder.AddSystem(ArmPlant_[None](arm_dynamics))
    # Setup visualization
    visualizer = builder.AddSystem(Arm2DVisualizer(arm_dynamics, show=False))
    builder.Connect(plant.get_output_port(), visualizer.get_input_port())

    diagram = builder.Build()
    print("Finished Building")
    print(f"Frame rate is set to: {1/frame_rate} fps")

    # Set up a simulator to run this diagram
    simulator = Simulator(diagram)
    context = simulator.get_mutable_context()

    visualizer.start_recording()
    for t in np.hstack((np.arange(x_traj.start_time(),
                                x_traj.end_time(),
                                frame_rate),
                        x_traj.end_time())):
        context.SetTime(t)
        x = x_traj.value(t)
        context.SetContinuousState(x)
        diagram.Publish(context)
    
    t_end = x_traj.end_time()
    x_end = x_traj.value(t_end)
    for _ in range(int(post_buffer*32)):
        context.SetTime(t_end)
        context.SetContinuousState(x_end)
        diagram.Publish(context)

    ani = visualizer.get_recording_as_animation()
    if show:
        display(HTML(ani.to_jshtml()))
    return ani

# Trajectory Optimization

In [ ]:
options = SolverOptions()
#options.SetOption(CommonSolverOption.kPrintFileName, "./debugOutput.txt")
snopt= SnoptSolver().solver_id()
options.SetOption(snopt, 'Iterations Limits', 1e5) # 1e5
options.SetOption(snopt, 'Major Feasibility Tolerance', 1e-6) #1e-6
options.SetOption(snopt, 'Function Precision', 1e-6) 
options.SetOption(snopt, 'Major Optimality Tolerance', np.sqrt(1e-6)) #2e-6 # sqrt of function precision
options.SetOption(snopt, 'Major Iterations Limit', 1000) 
options.SetOption(snopt, 'Superbasics limit', 2000) #209

In [ ]:
ArmPlant = ArmPlant_[None]  # Default instantiation
J = 0.0035/18.75**2
k_t = 0.18
nu = 0.0015
R = 2.0
L = 0

In [ ]:
arm_dynamics = ArmDynamics(0.145,0.125, 4*24, verbose=True)

In [ ]:
builder = DiagramBuilder()
arm = builder.AddSystem(ArmPlant(arm_dynamics))
builder.ExportInput(arm.get_input_port(0), 'input')

diagram = builder.Build()
context = diagram.CreateDefaultContext()
diagram.Publish(context)

In [ ]:
x_traj_list, u_traj_list, v_traj_list = [], [], []
x_traj, u_traj, v_traj = None, None, None
best_cost = np.inf
duration_bounds = [0.1,1]
print("Start Iterations")
for N in [11, 21,41]:
    arm_context = arm.CreateDefaultContext()
    print("-----------------------------------")
    print(f"min_dt: {duration_bounds[0]/(N-1)}")
    print(f"max_dt: {duration_bounds[1]/(N-1)}")
    print(f"N: {N}")
    dircol = DirectCollocation(arm,
                            arm_context,
                            num_time_samples=N,
                            minimum_timestep=duration_bounds[0]/N,
                            maximum_timestep=duration_bounds[1]/N)
 
    prog = dircol.prog()
    dircol.AddEqualTimeIntervalsConstraints()
    Vs = dircol.NewSequentialVariable(2, 'voltage')
    actuation_limit = 12.0  # V.
    #actuation_limit = 5.0  # V.
    u = dircol.input()
    for i in range(len(u)):
        dircol.AddConstraintToAllKnotPoints(-actuation_limit <= Vs[i])
        dircol.AddConstraintToAllKnotPoints(Vs[i] <= actuation_limit)
    
    # intial torque is 0
    ui = dircol.input(0)
    for i in range(len(ui)):
        dircol.AddConstraintToAllKnotPoints(ui[i] == 0)
        

    # joint position constraints 
    s = dircol.state()
    dircol.AddConstraintToAllKnotPoints(s[0] <= arm.PositionUpperLimit[0])
    dircol.AddConstraintToAllKnotPoints(s[1] <= arm.PositionUpperLimit[1])
    dircol.AddConstraintToAllKnotPoints(s[2] <= arm.PositionUpperLimit[2])
    dircol.AddConstraintToAllKnotPoints(s[0] >= arm.PositionLowerLimit[0])
    dircol.AddConstraintToAllKnotPoints(s[1] >= arm.PositionLowerLimit[1])
    dircol.AddConstraintToAllKnotPoints(s[2] >= arm.PositionLowerLimit[2])
    
    #initial joint velocity is 0
    si = dircol.initial_state()
    dircol.AddConstraintToAllKnotPoints(si[3] == 0)
    dircol.AddConstraintToAllKnotPoints(si[4] == 0)
    dircol.AddConstraintToAllKnotPoints(si[5] == 0)
    
    
    # initial joint angle of the wrist joint is 0
    prog.AddConstraint(si[2] == 0)
    
    Vsi = dircol.GetSequentialVariableAtIndex('voltage',0)
    prog.AddConstraint(ui[0] == - k_t * (Vsi[0] - k_t*si[3])/R + nu*si[3])
    prog.AddConstraint(ui[1] == - k_t * (Vsi[1] - k_t*si[4])/R + nu*si[4])
    # Motor dynamics limit 
    for i in range(1,N):
        si = dircol.state(i)
        si_1 = dircol.state(i-1)
        ui = dircol.input(i)
        Vsi = dircol.GetSequentialVariableAtIndex('voltage',i)
        hi = dircol.timestep(i-1)
        prog.AddConstraint(ui[0] == J * (si[3] - si_1[3])/hi[0] - k_t * (Vsi[0] - k_t*si[3])/R + nu*si[3])
        prog.AddConstraint(ui[1] == J * (si[4] - si_1[4])/hi[0] - k_t * (Vsi[1] - k_t*si[4])/R + nu*si[4])


    qf = dircol.final_state()
    prog.AddCost(arm_dynamics.EvalCost, vars = qf) 

    if x_traj is None:
        initial_x_traj = PiecewisePolynomial.FirstOrderHold([0., duration_bounds[1]], np.hstack([np.array([-3.14,1.57,0, 0, 0,0])[:,None],
                                                            np.array([3.14,0,-1.57, 10,10,10])[:,None]]))
        dircol.SetInitialTrajectory(PiecewisePolynomial(), initial_x_traj)
    
        for n in range(N):
            prog.SetInitialGuess(dircol.state(n), initial_x_traj.value(duration_bounds[1]*n/(N-1)))
    else:
        dircol.SetInitialTrajectory(PiecewisePolynomial(), x_traj)
        #for n in range(N):
        #    prog.SetInitialGuess(dircol.state(n), initial_x_traj.value(duration_bounds[1]*n/(N-1)))
        
    solver = SnoptSolver()

    result = solver.Solve(prog, solver_options=options)
    if not result.is_success():
        print("failed")
        break
    
    print(f"success: {result.is_success()}")
    if result.is_success():
        # reconstruct u_traj
        voltage_command = []
        for i in range(N):
            voltage_command.append(result.GetSolution(dircol.GetSequentialVariableAtIndex('voltage',i)))
        breaks = result.GetSolution(dircol.timestep(0))*np.arange(0,N,1)
        v_traj = PiecewisePolynomial.FirstOrderHold(breaks, np.vstack(voltage_command).T)
        
        x_traj_list.append(dircol.ReconstructStateTrajectory(result))
        u_traj_list.append(dircol.ReconstructInputTrajectory(result))
        v_traj_list.append(v_traj)
        
        visualize_arm_trajectory(x_traj_list[-1], frame_rate=1/460)
        plot_traj(x_traj_list[-1], u_traj_list[-1], v_traj_list[-1])
        #plt.show()
    
        x_traj = x_traj_list[-1]
        u_traj = u_traj_list[-1]
        v_traj = v_traj_list[-1]


### Visualize traj opt results

In [ ]:
ani = visualize_arm_trajectory(arm_dynamics, x_traj, frame_rate=1/800.)
fig = plot_traj(arm_dynamics, x_traj, u_traj, v_traj)

In [ ]:
# save animation
ani.save(f'data/our_optt_5v_limited.gif', writer='imagemagick', fps=32)
# save plot
fig.savefig(f'data/our_optt_5v_limited.png')

### compute voltage commands for embed

In [ ]:
release_time = u_traj.end_time()
q_init = np.round(x_traj.value(0).T,3)[0]
ts = np.linspace(v_traj.start_time(), v_traj.end_time(),41)
v_commands = []
u_commands = []
for t in ts:
    v_commands.append(v_traj.value(t))
    u_commands.append(u_traj.value(t))
dt = ts[1]-ts[0]
v_commands = np.round(np.hstack(v_commands),3)
u_commands = np.round(np.hstack(u_commands),3)
print(f'Release time: {release_time}')
print(f'Inital Joint Positions: {q_init[:3]}')
print(f'Voltage cmd 1: {v_commands[0]}')
print(f'Voltage cmd 2: {v_commands[1]}')
print(f'Torque cmd 1: {u_commands[0]}')
print(f'Torque cmd 2: {u_commands[1]}')
print(f'time delta: {dt}')

# Optimize for link length
l1 + l2 + l3 = 0.3

l3 = 0.5

l1,l2 > lfb1

In [ ]:
def optimize_for_link_length(l1, l2, k, simplify):
    arm_dynamics = ArmDynamics(l1, l2, k, simplify)

    builder = DiagramBuilder()
    arm = builder.AddSystem(ArmPlant(arm_dynamics))
    builder.ExportInput(arm.get_input_port(0), 'input')

    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    diagram.Publish(context)
    x_traj_list, u_traj_list, v_traj_list = [], [], []
    x_traj, u_traj, v_traj = None, None, None
    best_cost = np.inf
    duration_bounds = [0.1,1]
    for N in [11,21,41]:
        print(f'Iteration {N}')
        arm_context = arm.CreateDefaultContext()
        dircol = DirectCollocation(arm,
                                arm_context,
                                num_time_samples=N,
                                minimum_timestep=duration_bounds[0]/N,
                                maximum_timestep=duration_bounds[1]/N)
    
        prog = dircol.prog()
        Vs = dircol.NewSequentialVariable(2, 'voltage')
        #actuation_limit = 2.0  # N*m.
        actuation_limit = 12.0  # V.
        u = dircol.input()

        for i in range(len(u)):
            dircol.AddConstraintToAllKnotPoints(-actuation_limit <= Vs[i])
            dircol.AddConstraintToAllKnotPoints(Vs[i] <= actuation_limit)
    
        # intial torque is 0
        ui = dircol.input(0)
        for i in range(len(ui)):
            dircol.AddConstraintToAllKnotPoints(ui[i] == 0)
        

        # joint position constraints 
        s = dircol.state()
        dircol.AddConstraintToAllKnotPoints(s[0] <= arm.PositionUpperLimit[0])
        dircol.AddConstraintToAllKnotPoints(s[1] <= arm.PositionUpperLimit[1])
        dircol.AddConstraintToAllKnotPoints(s[2] <= arm.PositionUpperLimit[2])
        dircol.AddConstraintToAllKnotPoints(s[0] >= arm.PositionLowerLimit[0])
        dircol.AddConstraintToAllKnotPoints(s[1] >= arm.PositionLowerLimit[1])
        dircol.AddConstraintToAllKnotPoints(s[2] >= arm.PositionLowerLimit[2])

        #initial joint velocity is 0
        si = dircol.initial_state()
        dircol.AddConstraintToAllKnotPoints(si[3] == 0)
        dircol.AddConstraintToAllKnotPoints(si[4] == 0)
        dircol.AddConstraintToAllKnotPoints(si[5] == 0)
    
    
        # initial joint angle of the wrist joint is 0
        prog.AddConstraint(si[2] == 0)
    
        Vsi = dircol.GetSequentialVariableAtIndex('voltage',0)
        prog.AddConstraint(ui[0] == - k_t * (Vsi[0] - k_t*si[3])/R + nu*si[3])
        prog.AddConstraint(ui[1] == - k_t * (Vsi[1] - k_t*si[4])/R + nu*si[4])
        # Motor dynamics limit 
        for i in range(1,N):
            si = dircol.state(i)
            si_1 = dircol.state(i-1)
            ui = dircol.input(i)
            Vsi = dircol.GetSequentialVariableAtIndex('voltage',i)
            hi = dircol.timestep(i-1)
            prog.AddConstraint(ui[0] == J * (si[3] - si_1[3])/hi[0] - k_t * (Vsi[0] - k_t*si[3])/R + nu*si[3])
            prog.AddConstraint(ui[1] == J * (si[4] - si_1[4])/hi[0] - k_t * (Vsi[1] - k_t*si[4])/R + nu*si[4])

        qf = dircol.final_state()
        prog.AddCost(arm_dynamics.EvalCost, vars = qf) 

        if x_traj is None:
            initial_x_traj = PiecewisePolynomial.FirstOrderHold([0., duration_bounds[1]], np.hstack([np.array([-3.14,-1.57,0, 0, 0,0])[:,None],
                                                                np.array([3.14,0,1.57, 10,10,10])[:,None]]))
            dircol.SetInitialTrajectory(PiecewisePolynomial(), initial_x_traj)
        
            for n in range(N):
                prog.SetInitialGuess(dircol.state(n), initial_x_traj.value(duration_bounds[1]*n/(N-1)))
        else:
            dircol.SetInitialTrajectory(PiecewisePolynomial(), x_traj)
            #for n in range(N):
            #    prog.SetInitialGuess(dircol.state(n), initial_x_traj.value(duration_bounds[1]*n/(N-1)))

        solver = SnoptSolver()

        result = solver.Solve(prog, solver_options=options)
        if not result.is_success():
            break
        
        if result.is_success():
            # reconstruct u_traj
            voltage_command = []
            for i in range(N):
                voltage_command.append(result.GetSolution(dircol.GetSequentialVariableAtIndex('voltage',i)))
            breaks = result.GetSolution(dircol.timestep(0))*np.arange(0,N,1)
            v_traj = PiecewisePolynomial.FirstOrderHold(breaks, np.vstack(voltage_command).T)
            
            x_traj_list.append(dircol.ReconstructStateTrajectory(result))
            u_traj_list.append(dircol.ReconstructInputTrajectory(result))
            v_traj_list.append(v_traj)
            
            x_traj = x_traj_list[-1]
            u_traj = u_traj_list[-1]
            v_traj = v_traj_list[-1]
    
    data = []
    for x_traj, u_traj, v_traj in zip(x_traj_list, u_traj_list, v_traj_list):
        t = np.linspace(x_traj.start_time(), x_traj.end_time(), int(1e4))
        x_log = np.array([x_traj.value(i) for i in t]).T
        u_log = np.array([u_traj.value(i) for i in t]).T
        v_log = np.array([v_traj.value(i) for i in t]).T
        
        data.append((t, x_log, u_log, v_log))
        
    return l1, l2, k, data, arm_dynamics.neg_speed.to_string()

In [ ]:
total_length = 0.32 - 0.05
l1s = np.arange(0.030, total_length-0.025, 0.005)
l2s = total_length - l1s
ks = 24*np.arange(2,11,2)

In [ ]:
inputs = []
for k in ks:
    for l1, l2 in zip(l1s,l2s):
        inputs.append((l1,l2,k))
len(inputs)

### Run optimizations in Parallel

In [ ]:
with mp.Pool(processes = mp.cpu_count()) as pool:
    outputs = pool.starmap(optimize_for_link_length, inputs)

### Find best results

In [ ]:
def eval_speed(l1, l2, k, data, neg_speed_string):
    t, x_log, u_log, v_log = data[-1]
    q_last = x_log.T[-1,:,0]
    th1, th2, th3, dth1, dth2, dth3 = q_last
    speed = -eval(neg_speed_string)
    return speed

In [ ]:
upper_speed_limit = 150
plot_l1 = []
plot_k = []
plot_speed = []
for l1, l2, k, data, neg_speed_string in outputs:
    if len(data) == 3:
        t, x_log, u_log, v_log = data[-1]
        q_last = x_log.T[-1,:,0]
        th1, th2, th3, dth1, dth2, dth3 = q_last
        speed = -eval(neg_speed_string)
        plot_l1.append(l1)
        plot_k.append(k)
        if speed < upper_speed_limit:
            plot_speed.append(speed)
        else:
            print(l1,k, speed)
            print(np.abs(q_last[3:]))

In [ ]:
best_speed_idx = sorted(filter(lambda x: len(outputs[x][3]) == 3,range(len(outputs))), key = lambda x: -eval_speed(*outputs[x]))

In [ ]:
L1, K = np.meshgrid(plot_l1, plot_k)
SPEED = -np.ones(K.shape)
for l1, k, speed in zip(plot_l1, plot_k, plot_speed):
    SPEED[np.bitwise_and(L1 == l1, K == k)] = speed

fig = plt.figure(figsize=(15,6))
ax = fig.add_subplot(1,1,1)
CS = ax.contourf(L1, K, SPEED, cmap = cm.plasma)
for i, (l1, l2, k, data, neg_speed_string) in enumerate(outputs):
    if len(data)==3 and eval_speed(*outputs[i])>0:
        ax.scatter(l1, k, color='black')
        if i in best_speed_idx[:8]:
            ax.scatter(l1, k, color='black', s=500,facecolors='none')  
    
ax.set_xlabel('$L_1$ (m)')
ax.set_ylabel('k (N/m)')
cbar = fig.colorbar(CS)
cbar.ax.set_ylabel('speed (m/s)')

In [ ]:
def plot_traj_scrappy(t, x_log, u_log, v_log):
    velocities = []
    for q in x_log.T:
        th1, th2, th3, dth1, dth2, dth3 = q
        speed = -eval(neg_speed_string)
        velocities.append(speed)
    velocities = np.array(velocities) 

    fig, ax = plt.subplots(2, 2, figsize=(15,12), facecolor='white')
    #joint velocities
    ax[1,1].plot(t, x_log.T[:,3], label='$\dot{q_1}$')
    ax[1,1].plot(t, x_log.T[:,4], label='$\dot{q_2}$')
    ax[1,1].plot(t, x_log.T[:,5], label='$\dot{q_3}$')
    ax[1,1].set_title("Joint Velocities")
    ax[1,1].set_xlabel("t (s)")
    ax[1,1].set_ylabel("$\dot{\\theta}$ (rad/s)")
    ax[1,1].legend()
    
    ax[0,1].plot(t, velocities, label='$\sqrt{\dot{x}^2 + \dot{y}^2}$')
    ax[0,1].set_title("Endeffector Speed")
    ax[0,1].set_xlabel("t (s)")
    ax[0,1].set_ylabel("speed (m/s)")
    ax[0,1].legend()

    ax[0,0].plot(t, v_log.T[:,0], label='$V_1$')
    ax[0,0].plot(t, v_log.T[:,1], label='$V_2$')
    ax[0,0].set_title("Motor Voltages")
    ax[0,0].set_xlabel("t (s)")
    ax[0,0].set_ylabel("Voltage (V)")
    ax[0,0].legend()
    
    ax[1,0].plot(t, u_log.T[:,0], label='$\\tau_1$')
    ax[1,0].plot(t, u_log.T[:,1], label='$\\tau_2$')
    ax[1,0].set_title("Motor Torques")
    ax[1,0].set_xlabel("t (s)")
    ax[1,0].set_ylabel("Torque (Nm)")
    ax[1,0].legend()

    return fig


def visualize_arm_trajectory_scrappy(l1,l2,k, t_list, x_list, frame_rate = 1/60.0, post_buffer=2.0, show=False):
    builder = DiagramBuilder()
    arm_dynamics = ArmDynamics(l1,l2,k,True)
    plant = builder.AddSystem(ArmPlant(arm_dynamics))
    # Setup visualization
    visualizer = builder.AddSystem(Arm2DVisualizer(arm_dynamics, show=False))
    builder.Connect(plant.get_output_port(), visualizer.get_input_port())

    diagram = builder.Build()
    print("Finished Building")
    steps = int((len(t_list)/t_list[-1])*frame_rate)
    print(f"Frame rate is set to: {1/frame_rate} fps")

    # Set up a simulator to run this diagram
    simulator = Simulator(diagram)
    context = simulator.get_mutable_context()

    visualizer.start_recording()
    for t,x in zip(t_list[::steps], x_list.T[::steps]):
        context.SetTime(t)
        context.SetContinuousState(x)
        diagram.Publish(context)
    for _ in range(int(post_buffer*32)):
        context.SetTime(t_list[-1])
        context.SetContinuousState(x_list.T[-1])
        diagram.Publish(context)

    ani = visualizer.get_recording_as_animation()
    if show:
        display(HTML(ani.to_jshtml()))
    return ani

In [ ]:
idx = best_speed_idx[0]
l1,l2,k,data,speed_string = outputs[idx]
print(l1,l2,k)
t_list, x_log, u_log,v_log = data[-1]
fig = plot_traj_scrappy(t_list, x_log, u_log, v_log)
ani = visualize_arm_trajectory_scrappy(l1,l2,k,t_list, x_log, 1/460.0)

### Save ALL GIFs and Plots

In [ ]:
for i, idx in enumerate(best_speed_idx):
    l1,l2,k,data,speed_string = outputs[idx]
    t_list, x_log, u_log,v_log = data[-1]
    speed = round(eval_speed(l1, l2, k, data, speed_string),2)
    name = f"{i}_{round(l1,3)}_{round(l2,3)}_{int(k)}_{speed}"
    #make folder
    os.mkdir(f'data/{name}')
    
    #save animation
    visualize_arm_trajectory_scrappy(l1,l2,k,t_list, x_log, 1/460.0).save(f'data/{name}/{name}.gif', writer='imagemagick', fps=32)
    
    #save figure
    fig = plot_traj_scrappy(t_list, x_log, u_log, v_log)
    fig.savefig(f'data/{name}/{name}.png')
    plt.close(fig)